<a href="https://colab.research.google.com/github/bipin-a/thesis-meng/blob/main/Robustness_Exp2_perturbations_dependancies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install textattack transformers sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.7/418.7 KB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.3/374.3 KB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 98.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 KB 15.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 KB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 18.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━

In [ ]:
import transformers 
import textattack

In [ ]:
import torch
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.goal_functions import UntargetedClassification

from textattack.attack_recipes import (
    TextFoolerJin2019,
    HotFlipEbrahimi2017,
    BAEGarg2019
)
from datasets import load_dataset


In [ ]:
torch.cuda.is_available() 

True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
perturbed_data_root = '/content/drive/MyDrive/meng_thesis/code/data/perturbed_data'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Experiment 2: 
- Hypothesis: The perturbations of white box adversarial attacks dependant on the victim model.

    1. Select white box adv attack 
    2. Select a dataset 
    3. Test a range of Victim Models
    4. Generate list of perturbed datasets for each victim model
    5. Compare overlap of inputs across lists datasets  
    6. Repeat steps 1-6 with different white box adv attacks 


In [ ]:
# Get huggingface models and tokenizer

checkpoint_names = [
    "textattack/distilbert-base-uncased-imdb",
    "textattack/albert-base-v2-imdb",
    "textattack/roberta-base-imdb"
    ]
model_names = [
    'distilbert-base-uncased',
    'albert-base-v2',
    'roberta-base-imdb'
    ]
dataset_name = 'imdb'

models = [transformers.AutoModelForSequenceClassification.from_pretrained(c) for c in checkpoint_names]
tokenizers = [transformers.AutoTokenizer.from_pretrained(c) for c in checkpoint_names]
model_wrappers = [HuggingFaceModelWrapper(m, t) for m,t in zip(models,tokenizers)]

hg_dataset = load_dataset(dataset_name, split="test[:10]") 


Some weights of the model checkpoint at textattack/roberta-base-imdb were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
dataset = textattack.datasets.Dataset( [(i.get("text") , i.get("label")) for i in hg_dataset] )

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(hg_dataset)

,text,label
0,I love sci-fi and am willing to put up with a ...,0
1,"Worth the entertainment value of a rental, esp...",0
2,its a totally average film with a few semi-alr...,0
3,STAR RATING: ***** Saturday Night **** Friday ...,0
4,"First off let me say, If you haven't enjoyed a...",0
5,I had high hopes for this one until they chang...,0
6,Isaac Florentine has made some of the best wes...,0
7,"It actually pains me to say it, but this movie...",0
8,"Technically I'am a Van Damme Fan, or I was. th...",0
9,"Honestly awful film, bad editing, awful lighti...",0


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
parallel_flag = bool(torch.cuda.device_count())
parallel_flag

True

In [ ]:
attacks = [
    TextFoolerJin2019, 
    # HotFlipEbrahimi2017,
    BAEGarg2019
]

attack_names = [
    "TextFoolerJin2019", 
    # "HotFlipEbrahimi2017",
    "BAEGarg2019"
]

In [ ]:
for attack_model, attack_name in zip(attacks, attack_names):
  for model_wrapper, model_name in zip(model_wrappers, model_names):
    
    print(model_name,attack_name)
    attack = attack_model.build(model_wrapper)

    attack_args = textattack.AttackArgs(
        parallel = parallel_flag,
        num_examples=5,
        csv_coloring_style = 'html',
        log_to_csv=f"{perturbed_data_root}/{attack_name}{model_name}_html.csv",
        # checkpoint_interval=5,
        # checkpoint_dir="checkpoints",
        disable_stdout=True
    )

    attacker = textattack.Attacker(attack, dataset, attack_args)
    attacker.attack_dataset()


textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path /content/drive/MyDrive/meng_thesis/code/data/perturbed_data/TextFoolerJin2019distilbert-base-uncased_html.csv
textattack: Running 1 worker(s) on 1 GPU(s).


distilbert-base-uncased TextFoolerJin2019


textattack: Worklist size: 5
textattack: Worklist candidate size: 5
[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5: 100%|██████████| 5/5 [01:16<00:00, 15.22s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 4      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 80.0%  |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 6.5%   |
| Average num. words per input: | 215.6  |
| Avg num queries:              | 546.75 |
+-------------------------------+--------+


textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path /content/drive/MyDrive/meng_thesis/code/data/perturbed_data/TextFoolerJin2019albert-base-v2_html.csv
textattack: Running 1 worker(s) on 1 GPU(s).


...


+-------------------------------+---------+
| Attack Results                |         |
+-------------------------------+---------+
| Number of successful attacks: | 3       |
| Number of failed attacks:     | 0       |
| Number of skipped attacks:    | 2       |
| Original accuracy:            | 60.0%   |
| Accuracy under attack:        | 0.0%    |
| Attack success rate:          | 100.0%  |
| Average perturbed word %:     | 10.04%  |
| Average num. words per input: | 215.6   |
| Avg num queries:              | 1112.67 |
+-------------------------------+---------+


textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path /content/drive/MyDrive/meng_thesis/code/data/perturbed_data/TextFoolerJin2019roberta-base-imdb_html.csv
textattack: Running 1 worker(s) on 1 GPU(s).



roberta-base-imdb TextFoolerJin2019


textattack: Worklist size: 5
textattack: Worklist candidate size: 5
[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:  20%|██        | 1/5 [05:44<22:56, 344.05s/it]

KeyboardInterrupt: ignored

In [ ]:
import pandas as pd

In [ ]:
model_names = ['distilbert-base-uncased','albert-base-v2']
df1 = pd.read_csv(f"{perturbed_data_root}/{model_names[0]}_html.csv")[['original_text','perturbed_text']]
df2 = pd.read_csv(f"{perturbed_data_root}/{model_names[1]}_html.csv")[['perturbed_text']]

df1 = df1.rename(columns={'perturbed_text':'perturbed_text_distilbert-base'})
df2 = df2.rename(columns={'perturbed_text':'perturbed_text_albert-base'})

main = pd.concat([df1,df2],axis=1)

In [ ]:
main.head(2).to_markdown()

'|    | original_text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

# TODO:
- Measure Perplexity and other metrics of the perturbed data

In [ ]:
https://huggingface.co/docs/transformers/perplexity

In [ ]:
HG_datasets = {}
dfs = {}

for dataset_name in dataset_names:
  df = pd.read_csv(f"{root}/data/{dataset_name}.csv")
  _dataset = df[["perturbed_text","ground_truth_output"]]
  dfs[dataset_name] = _dataset

  _dataset = _dataset.rename({"perturbed_text":"text", 
                              "ground_truth_output":"labels"}
                             ,axis=1)
  
  dataset = HG_Dataset.from_pandas(_dataset)
  HG_datasets[dataset_name] = dataset

In [ ]:
for name, df in dfs.items():
  print(f"name: {df.ground_truth_output.value_counts()}")

name: 1    3000
0    3000
Name: ground_truth_output, dtype: int64
name: 1    3000
0    3000
Name: ground_truth_output, dtype: int64
name: 1    2002
0    1998
Name: ground_truth_output, dtype: int64
name: 1    504
0    496
Name: ground_truth_output, dtype: int64
name: 1    504
0    496
Name: ground_truth_output, dtype: int64
name: 1    501
0    499
Name: ground_truth_output, dtype: int64


## textattack/bert-base-uncased-imdb

In [ ]:
metric_results = {}

for hg_name in HG_datasets:
  for checkpoint_name in checkpoint_names:


    torch.cuda.empty_cache()
    dataset = HG_datasets[hg_name]
    print(checkpoint_name, hg_name)

    clf_metrics = evaluate.combine(["accuracy", "recall", "precision", "f1"])

    model = transformers.AutoModelForSequenceClassification.from_pretrained(checkpoint_name)
    model_tokenizer = transformers.AutoTokenizer.from_pretrained(checkpoint_name)
    model.to(device)

    tokenized_datasets_all = dataset.map(
        lambda examples : model_tokenizer(examples["text"], 
                                  padding="max_length",
                                  truncation=True)
        )
    
    tokenized_datasets = tokenized_datasets_all.remove_columns(['text'])
    tokenized_datasets.set_format("torch")

    test_dataloader = DataLoader(tokenized_datasets,
                              shuffle=True, 
                              batch_size=8)

    model.eval()
    for batch in test_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        clf_metrics.add_batch(predictions=predictions, references=batch["labels"])

    m_res = clf_metrics.compute()
    print(m_res)

    metric_results[(hg_name,checkpoint_name)] = m_res
    print("\n \n ")


textattack/bert-base-uncased-imdb log_A2TYoo2021_0.05


  0%|          | 0/6000 [00:00<?, ?ex/s]

{'accuracy': 0.8575, 'recall': 0.805, 'precision': 0.8994413407821229, 'f1': 0.8496042216358839}

 
 
textattack/distilbert-base-uncased-imdb log_A2TYoo2021_0.05


Downloading:   0%|          | 0.00/485 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

...

{'accuracy': 0.802, 'recall': 0.7584830339321357, 'precision': 0.8315098468271335, 'f1': 0.7933194154488518}

 
 
textattack/albert-base-v2-imdb log_BAEGarg2019_attack_BERT_all


  0%|          | 0/1000 [00:00<?, ?ex/s]

{'accuracy': 0.84, 'recall': 0.8243512974051896, 'precision': 0.8515463917525773, 'f1': 0.8377281947261663}

 
 
/content/drive/Shareddrives/BipinA/EE8227_secure_ml/Project/adv_trained_model/best_model log_BAEGarg2019_attack_BERT_all


  0%|          | 0/1000 [00:00<?, ?ex/s]

{'accuracy': 0.819, 'recall': 0.810379241516966, 'precision': 0.8252032520325203, 'f1': 0.8177240684793554}

 
 
